##### TODO

- simulate buying stocks (buy/sell decisions)


In [1]:
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt

from tensorflow import keras
import numpy as np
import datetime

In [2]:
def sampleDataset(price_changes, num_samples, interval_len, extension):
    # creates samples of data from a dataset
    # returns the intervals, and the target price changes

    price_intervals = np.ndarray((num_samples, interval_len))
    target_changes = np.ndarray((num_samples, extension))
    for x in range(num_samples):
        # print(x)
        # start_index = np.random.randint(len(price_changes) - interval_len - extension)
        start_index = round(x / (len(price_changes) - interval_len - extension))

        price_intervals[x] = price_changes[start_index:start_index + interval_len]
        # print(price_intervals[x])
        target_changes[x] = price_changes[start_index + interval_len:start_index + interval_len + extension]
        # print(target_changes[x])
        # print()
    
    return price_intervals, target_changes

In [ ]:
# Calculate hourly price changes for the ticker.
import pickle

df = pd.read_csv("a.csv")

TICKER = list(df.loc[:, "Symbol"])[:20]
# TICKER = ["MSFT", "AAPL", "NVDA", "MMM", "ABBV", "ADBE", "GOOGL", "T", "CMCSA", "DIS", "EA", "FOX", "META", "NFLX", "WBD", "ABNB", "TSLA", "AMZN", "DBX"]
# TICKER = ["MSFT"]

# print(TICKER[96])

DATASET_SIZE = 3500 # Number of samples PER TICKER
INTERVAL = 7
EXTENSION = 1

# fetch hists
try:
    with open("hists.dat", "rb") as f:
        hists = pickle.load(f) 
except:
    hists = []
    for x in range(len(TICKER)):
        print("LOADING TICKER " + str(x) + "/" + str(len(TICKER)), end="\r")
        hist = yf.Ticker(TICKER[x]).history(interval="1h", period="2y")
        if not hist.empty and np.sum(np.isnan(hist.loc[:, "Open"].to_numpy())) == 0:
            hists.append(hist)

price_intervals = np.ndarray((len(hists), DATASET_SIZE, INTERVAL, 1))
target_changes = np.ndarray((len(hists), DATASET_SIZE, EXTENSION))
for x in range(len(hists)):
    open_prices = hists[x].loc[:, "Open"].to_numpy()[:-1]
    next_prices = hists[x].loc[:, "Open"].to_numpy()[1:]

    ''' top: $ change, centered at 0
        bottom: net change'''
    
    price_changes = (next_prices - open_prices) / open_prices * 100
    # price_changes = next_prices - open_prices
    # print(price_changes.shape)
    
    '''==='''

    # print(open_prices.shape)
    # print(next_prices.shape)
    # print(open_prices[:5])
    # print(next_prices[:5])
    # print(price_changes[:5])
    
    intervals, target_changes[x] = sampleDataset(price_changes, DATASET_SIZE, INTERVAL, EXTENSION)
    # if x == 0:
    #     print(intervals[0])
    # print(intervals.shape)
    price_intervals[x] = intervals.reshape(intervals.shape[0], intervals.shape[1], 1)
    # print(price_intervals[x, 0, 0, :5])

    if(np.sum(np.isnan(price_intervals[x]))):
        print("NAN: " + str(x))
        print(np.where(np.isnan(price_intervals[x])))
        # print(intervals[0])
        print(open_prices)
        print(price_intervals[x][0][0])
        display(hists[x])

price_intervals = price_intervals.reshape((len(hists) * DATASET_SIZE, INTERVAL, 1))
target_changes = target_changes.reshape((len(hists) * DATASET_SIZE, EXTENSION))

with open("hists.dat", "wb") as f:
    pickle.dump(hists, f)

def normalize(data):
    return (data - np.mean(data)) / np.std(data)

# price_intervals = normalize(price_intervals)
# target_changes = normalize(target_changes)

np.set_printoptions(suppress=True)
# print()
# print(price_intervals.shape)
# print(target_changes.shape)
# print(price_intervals[0, :, 0])
# print(target_changes[0])
# plt.hist(price_intervals[:, :, 0])
# plt.show()

# print("===")
# print(hists[0][:9]["Open"])
# print(price_intervals[0, 0, 0])
# print(target_changes[0])


In [ ]:
# Create LSTM model
LEARNING_RATE = 0.001
REG_FACTOR = 0 
VAL_RATIO = 0

# Define the LSTM model
model = keras.Sequential()
model.add(keras.layers.Input((INTERVAL, 1)))
# model.add(keras.layers.LSTM(units=64, kernel_regularizer=keras.regularizers.l2(REG_FACTOR)))
# model.add(keras.layers.Dense(units=64, activation="relu", kernel_regularizer=keras.regularizers.l2(REG_FACTOR)))
# model.add(keras.layers.Dense(units=EXTENSION, activation="linear", kernel_regularizer=keras.regularizers.l2(REG_FACTOR)))
# model.add(keras.layers.LSTM(units=32, input_shape=(INTERVAL, 1), return_sequences=True))
# model.add(keras.layers.Dropout(0.1))
# model.add(keras.layers.LSTM(units=32, input_shape=(INTERVAL, 1)))
# model.add(keras.layers.Dropout(0.1))
# model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=64, activation="relu"))
model.add(keras.layers.Dense(units=EXTENSION, activation="linear"))

# Compile the model
model.compile(loss=keras.losses.MSE, optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE))

model.build((None, INTERVAL, 1))
model.summary()

In [ ]:
num_epochs = 10
batch_size = 384

# print(model.pjuredict(np.expand_dims(price_intervals[0], axis=0)))
# Train the model
model.fit(price_intervals, target_changes, epochs=num_epochs, batch_size=batch_size, validation_split=VAL_RATIO)

# print(model.predict(np.expand_dims(price_intervals[0], axis=0)))

# Evaluate the model
# print(price_intervals.shape)
# print(target_changes.shape)
metrics = model.evaluate(price_intervals, target_changes)

# Make predictions
predictions = model.predict(np.expand_dims(price_intervals[0], axis=0))
print("PREDICTION: " + str(predictions[0,0]))   
print("ACTUAL: " + str(target_changes[0,0]))
print("ERROR (%): " + str((predictions[0,0] - target_changes[0,0]) / target_changes[0,0] * 100))

predictions = model.predict(price_intervals[:10,])
print(predictions)

In [ ]:
from numpy import NaN


def get_samples(hists, interval, type="change", offset=0, steps=0):
    num_hists = len(hists)
    samples = np.ndarray((num_hists, interval + steps))
    timestamps = np.ndarray((num_hists, 2), dtype="datetime64[s]")
    for x in range(num_hists):
        hist = hists[x]

        hist_times = np.asarray(hist.index.astype(np.int64) / (10 ** 9), dtype="datetime64[s]")
        timestamps[x] = np.array((hist_times[-interval - offset], hist_times[-1 - offset + steps]))

        open_prices = hists[x].loc[:, "Open"].to_numpy()[:-1]
        next_prices = hists[x].loc[:, "Open"].to_numpy()[1:]

        price_changes = (next_prices - open_prices) / open_prices * 100

        if type == "change":
            samples[x] = price_changes[-interval - offset:price_changes.shape[0] - offset + steps]
        elif type == "open":
            samples[x] = open_prices[-interval - offset:price_changes.shape[0] - offset + steps]

    return samples, timestamps

def format_datetime(timestamp):
    return timestamp.astype(datetime.datetime).strftime("%m/%d/%Y, %H:%M")

def plot(ticker, sample, timestamps, predictions=np.array((NaN,)), actual=np.array((NaN,)), actual_end=None, type="change", offset=0, change_error=False):
    print("=== " + ticker + ": " + type + " ===")
    print("Start: " + format_datetime(timestamps[0]))
    print("  End: " + format_datetime(timestamps[1]))
    
    sample = sample[offset:]
    actual = actual[offset:]

    fig = plt.figure(figsize=(12, 3))
    ax_sample = fig.add_subplot(141)
    ax_sample.plot(sample)
    ax_sample.set_title("Sample")

    if predictions[0] != NaN:
        # print(sample[-1])
        # print(sample)
        # print(predictions)
        # print(np.cumsum(predictions))

        
        if type != "change":
            predictions = sample[-1] * np.cumprod(predictions / 100 + 1)
            # np.cumsum(predictions) + sample[-1]
            # predictions = predictions + sample[-1]
            


        # print(predictions)


        forecast = np.concatenate((sample, predictions))
        ax_prediction = fig.add_subplot(142)
        ax_prediction.plot(forecast)
        ax_prediction.set_title("Prediction")
    else:
        print("plot(): predictions either NaN or not given")
        
    if actual[0] != NaN:
        print("Actual prices end: " + format_datetime(actual_end))
        ax_actual = fig.add_subplot(143)
        ax_actual.plot(actual)
        ax_actual.set_title("Actual")
    else:
        print("plot(): actual values either NaN or not given")

    if change_error:
        ax_error = fig.add_subplot(144)
        forecast = np.concatenate((sample, predictions))
        ax_error.plot(forecast - actual)
        ax_error.set_title("Error in prediction")

    plt.show()

def predict(model, samples, steps):
    '''FOR CHAINED PREDICTIONS'''
    predictions = np.ndarray((samples.shape[0], steps))

    # print(predictions.shape)
    # print(samples.shape)
    # print(np.expand_dims(samples, axis=1).shape)
    for x in range(steps):
        # print(model.predict(np.expand_dims(samples, axis=2), verbose=0).shape)
        predictions[:, x] = model.predict(np.expand_dims(samples, axis=2), verbose=0)[:, 0]

        samples[:, :INTERVAL - 1] = samples[:, 1:]
        samples[:, INTERVAL - 1] = predictions[:, x]
    return predictions

In [ ]:
STEPS = 7
BACK_OFFSET = 2
TYPE = "open"

input_sample, _ = get_samples(hists, INTERVAL, type="change", offset=STEPS)
samples, timestamps = get_samples(hists, INTERVAL, type=TYPE, offset=STEPS)
actuals, actuals_end = get_samples(hists, INTERVAL, type=TYPE, offset=STEPS, steps=STEPS)

# print(samples.shape)
# print(actuals.shape)

predictions = predict(model, np.copy(input_sample), STEPS)

# print(samples.shape)
# print(timestamps.shape)
# print(actuals.shape)
# print(actuals_end.shape)
# print(predictions.shape)
# print(predictions)
# print(predictions[0:2])

act_change, _ = get_samples(hists, INTERVAL, type="change", offset=STEPS, steps=STEPS)


for x in range(len(hists)):
    plot(TICKER[x], samples[x], timestamps[x], predictions=predictions[x], actual=actuals[x], actual_end=actuals_end[x][1], type=TYPE, offset=INTERVAL - BACK_OFFSET, change_error=True)
    # pred = predictions[x, :5]
    # acts = actuals[x, -STEPS:-STEPS + 5]
    # act_change = act_change[x, -STEPS:-STEPS + 5]

    


In [ ]:
# INDEX = 0

# # index = np.random.randint()
# sample = (hists[INDEX].loc[:, "Close"] - hists[INDEX].loc[:, "Open"])[:INTERVAL]
# sample = np.expand_dims(np.expand_dims(sample, axis=0), axis=0)

# original = (hists[INDEX].loc[:, "Close"] - hists[INDEX].loc[:, "Open"])[:INTERVAL].to_numpy()
# actual = (hists[INDEX].loc[:, "Close"] - hists[INDEX].loc[:, "Open"])[:INTERVAL + EXTENSION].to_numpy()
# # print(sample.shape)
# # print(sample)
# predictions = predict(model, sample)
# extended = np.concatenate((original, predictions))

# print("Original plot")
# plt.plot(original, color="blue")
# plt.show()

# print("Extrapolated plot")
# plt.plot(predictions, color="blue")
# plt.show()

# print("Actual plot")
# plt.plot(actual[INTERVAL:], color="blue")
# plt.show()

# print("Difference (pred - real) plot")
# plt.plot(predictions - actual[INTERVAL:], color="blue")
# plt.show()